### IMPORTAR LIBRERÍAS

In [1]:
import pandas as pd

### CARGA DE DATOS

In [2]:
address = 'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD/Basededatos/ACE.xlsx'
address2 = 'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD/Basededatos/ACE.xlsx'
# addressmarco = 'C:\\Users\\mticllacu\\OneDrive - LUZ DEL SUR S.A.A\\Archivos de Ronald Quispe Ocaña - ProyectoRyD\\Basededatos\\ACE.xlsx'
#df = pd.read_excel(address)
df = pd.read_excel(address2,header=1)

<!-- ### CARGA DE DATOS -->

### LIMPIEZA DE DATOS

In [3]:
df.columns = df.columns.str.replace(r"\n", " ", regex=True).str.strip()

# Eliminar columnas innecesarias
# En lugar de eliminar se deben extraer las columnas necesarias, ya que en otras ocasiones habrás más columnas innecesrias
cols_drop = [
    "Unnamed: 0",
    "Temp. De muestra (°C)",
    "Examen Visual (ASTM 1524)",
    "Gravedad Específica (ASTM D1298, g/mL)",
    "Humedad ambiente (%)",
]
df = df.drop(columns=cols_drop)

# Fechas y tensión
df["FECHA DE MUESTRA"] = pd.to_datetime(df["FECHA DE MUESTRA"])
df["TENSION"] = df["TENSION"].str.split("/").str[0]

# Renombrar columnas
rename_map = {
    "Contenido de humedad (ASTM D1533, ppm)": "HU",
    "Rigidez Dieléctrica (ASTM D1816, kV/2 mm)": "RD",
    "Tensión Interfacial (ASTM D971, mN/m)": "TIF",
    "Indice de neutralización (ASTM D974, mg KOH/g)": "AC",
    "Color (ASTM D1500)": "CO",
    "Factor de potencia a 25°C (ASTM D924, %)": "FP25",
    "Factor de potencia a 100°C (ASTM D924, %)": "FP100",
    "Contenido de inhibidor (ASTM D-2668)": "IO",
}
df = df.rename(columns=rename_map)

# Reordenar columnas
orden = ["SERIE", "TENSION", "FECHA DE MUESTRA", "FP25", "FP100", "HU", "AC", "TIF", "CO", "RD", "IO"]
df = df[orden]

# Asegurar numéricos
num_cols = ["FP25", "FP100", "HU", "AC", "TIF", "CO", "RD", "IO"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

In [4]:
df_full = df.copy()
df_full = df.drop(columns=["TENSION"]).copy()
def BD_DGA():
    return df_full
df_full.head()

,SERIE,FECHA DE MUESTRA,FP25,FP100,HU,AC,TIF,CO,RD,IO
0,146660T3,2022-05-12,1.3,7,91,0.06,41,6,31,0.31
1,146660T3,2018-09-17,0.3,9,41,0.07,43,2,15,0.25
2,146660T3,2015-08-19,0.3,1,77,0.06,31,3,38,0.26
3,146660T3,2013-02-19,0.6,5,63,0.02,29,0,39,0.35
4,364076,2022-06-18,0.1,4,43,0.08,41,3,46,0.21


### TRANSFORMACIÓN DE LOS DATOS

In [5]:
# =====================================
# DEFINICIÓN DE PUNTAJES IEEE
# =====================================
# Diccionario de reglas: {tensión: {parámetro: [(condición, puntaje), ...]}}

reglas = {
    "220": {
        "FP25": [(lambda x: x > 0.3, 5), (lambda x: 0.1 < x <= 0.3, 3), (lambda x: x <= 0.1, 1)],
        "FP100": [(lambda x: x > 4, 5), (lambda x: 3 < x <= 4, 3), (lambda x: x <= 3, 1)],
        "HU": [(lambda x: x > 30, 5), (lambda x: 20 < x <= 30, 3), (lambda x: x <= 20, 1)],
        "AC": [(lambda x: x > 0.10, 5), (lambda x: 0.05 < x <= 0.10, 3), (lambda x: x <= 0.05, 1)],
        "TIF": [(lambda x: x < 28, 5), (lambda x: 28 <= x < 32, 3), (lambda x: x >= 32, 1)],
        "CO": [(lambda x: x > 3.5, 5), (lambda x: x <= 3.5, 1)],
        "RD": [(lambda x: x < 45, 5), (lambda x: 45 <= x < 50, 3), (lambda x: x >= 50, 1)],
        "IO": [(lambda x: x < 0.1, 5), (lambda x: 0.1 <= x < 0.2, 3), (lambda x: x >= 0.2, 1)],
    },
    "60": {
        "FP25": [(lambda x: x > 0.3, 5), (lambda x: 0.1 < x <= 0.3, 3), (lambda x: x <= 0.1, 1)],
        "FP100": [(lambda x: x > 4, 5), (lambda x: 3 < x <= 4, 3), (lambda x: x <= 3, 1)],
        "HU": [(lambda x: x > 40, 5), (lambda x: 30 < x <= 40, 3), (lambda x: x <= 30, 1)],
        "AC": [(lambda x: x > 0.10, 5), (lambda x: 0.05 < x <= 0.10, 3), (lambda x: x <= 0.05, 1)],
        "TIF": [(lambda x: x < 28, 5), (lambda x: 28 <= x < 32, 3), (lambda x: x >= 32, 1)],
        "CO": [(lambda x: x > 3.5, 5), (lambda x: x <= 3.5, 1)],
        "RD": [(lambda x: x < 35, 5), (lambda x: 35 <= x < 40, 3), (lambda x: x >= 40, 1)],
        "IO": [(lambda x: x < 0.1, 5), (lambda x: 0.1 <= x < 0.2, 3), (lambda x: x >= 0.2, 1)],
    },
}

def puntaje_parametro(valor, tension, parametro):
    if pd.isna(valor):
        return float("nan")
    reglas_tension = reglas.get(str(tension), {})
    condiciones = reglas_tension.get(parametro, [])
    for condicion, puntaje in condiciones:
        if condicion(valor):
            return puntaje
    return 1  # valor por defecto

# Crear puntajes
parametros = num_cols
for p in parametros:
    df[f"Puntaje_{p}"] = df.apply(lambda row: puntaje_parametro(row[p], row["TENSION"], p), axis=1)


In [6]:
df.head(10)

,SERIE,TENSION,FECHA DE MUESTRA,FP25,FP100,HU,AC,TIF,CO,RD,IO,Puntaje_FP25,Puntaje_FP100,Puntaje_HU,Puntaje_AC,Puntaje_TIF,Puntaje_CO,Puntaje_RD,Puntaje_IO
0,146660T3,60,2022-05-12,1.3,7,91,0.06,41,6,31,0.31,5,5,5,3,1,5,5,1
1,146660T3,60,2018-09-17,0.3,9,41,0.07,43,2,15,0.25,3,5,5,3,1,1,5,1
2,146660T3,60,2015-08-19,0.3,1,77,0.06,31,3,38,0.26,3,1,5,3,3,1,3,1
3,146660T3,60,2013-02-19,0.6,5,63,0.02,29,0,39,0.35,5,5,5,1,3,1,3,1
4,364076,60,2022-06-18,0.1,4,43,0.08,41,3,46,0.21,1,3,5,3,1,1,1,1
5,364076,60,2021-08-06,0.8,10,27,0.15,19,1,28,0.10,5,5,1,5,5,1,5,3
6,364076,60,2017-12-06,1.4,4,45,0.00,39,1,36,0.06,5,3,5,1,1,1,3,5
7,364076,60,2012-10-19,1.4,2,44,0.08,16,4,25,0.03,5,1,5,3,5,5,5,5
8,230531-01,60,2024-07-16,0.8,0,91,0.01,21,5,29,0.11,5,1,5,1,5,5,5,3
9,230531-01,60,2021-01-29,0.4,4,71,0.03,36,3,22,0.30,5,3,5,1,1,1,5,1


In [7]:
# =====================================
# CÁLCULO DE ACE
# =====================================
pesos = {
    "Puntaje_FP25": 3,
    "Puntaje_FP100": 3,
    "Puntaje_HU": 4,
    "Puntaje_AC": 2,
    "Puntaje_TIF": 3,
    "Puntaje_CO": 1,
    "Puntaje_RD": 5,
    "Puntaje_IO": 1,
}

def calcular_ACE(row, pesos):
    valores = [row[col] * peso for col, peso in pesos.items() if pd.notna(row[col])]
    total_peso = sum(peso for col, peso in pesos.items() if pd.notna(row[col]))
    return sum(valores) / total_peso if total_peso > 0 else float("nan")

df["ACE"] = df.apply(lambda row: calcular_ACE(row, pesos), axis=1)
df_ACE = df[["SERIE", "FECHA DE MUESTRA", "ACE"]]

In [12]:
df_ACE

,SERIE,FECHA DE MUESTRA,ACE
0,146660T3,2022-05-12,4.090909
1,146660T3,2018-09-17,3.636364
2,146660T3,2015-08-19,2.909091
3,146660T3,2013-02-19,3.545455
4,364076,2022-06-18,2.181818
5,364076,2021-08-06,4.000000
6,364076,2017-12-06,3.181818
7,364076,2012-10-19,4.272727
8,230531-01,2024-07-16,4.000000
9,230531-01,2021-01-29,3.454545


In [8]:
# =====================================
# FUNCIÓN DE CONSULTA
# =====================================
def obtener_ACE(serie=None, tension=None, fecha_muestra=None):
    resultado = df_ACE.copy()
    if serie is not None:
        resultado = resultado[resultado["SERIE"] == serie]
    if tension is not None:
        resultado = resultado[resultado["TENSION"] == str(tension)]
    if fecha_muestra is not None:
        fecha_muestra = pd.to_datetime(fecha_muestra)
        resultado = resultado[resultado["FECHA DE MUESTRA"] == fecha_muestra]
        return resultado["ACE"].iloc[0] if not resultado.empty else float("nan")
    return resultado

### VALIDACIÓN DE LOS DATOS

In [9]:
# =====================================
# EJEMPLOS
# =====================================
print(obtener_ACE())  # toda la tabla
# print(obtener_ACE("D518293"))  # solo serie
# print(obtener_ACE("D518293", "220"))  # serie + tensión
# print(" ")
# print(obtener_ACE("D518293", "220", "2021-08-15"))  # serie + tensión + fecha

dataset = df_ACE

        SERIE FECHA DE MUESTRA       ACE
0    146660T3       2022-05-12  4.090909
1    146660T3       2018-09-17  3.636364
2    146660T3       2015-08-19  2.909091
3    146660T3       2013-02-19  3.545455
4      364076       2022-06-18  2.181818
5      364076       2021-08-06  4.000000
6      364076       2017-12-06  3.181818
7      364076       2012-10-19  4.272727
8   230531-01       2024-07-16  4.000000
9   230531-01       2021-01-29  3.454545
10  230531-01       2018-01-16  3.545455
11  230531-01       2013-01-03  3.090909
12     338118       2024-11-13  2.363636
13     338118       2017-06-15  3.727273
14     338118       2015-02-24  3.181818
15     338118       2014-08-27  4.181818
